# Write anomalies to clarify
In this notebook we write back the anomalies we have found to clarify.

In [1]:
import orchest
import numpy as np
from pyclarify import *
from utils import Anomaly

In [2]:
data = orchest.get_inputs()

clarify_data = data['clarify_data']
energy_consumption_dfs = clarify_data['energy_consumption']

anomalies = data['filtered_anomalies']['filtered_anomalies']

In [3]:
clarify_client = ClarifyClient("clarify-credentials.json")

In [4]:
def get_dataframe_with_anomalies(building_name):
    anomalies[building_name].sort(key=lambda x: x.start)

    c = None
    i = 0
    while i < len(anomalies[building_name]):
        if anomalies[building_name][i].start == c:
             anomalies[building_name].pop(i)
        else:
            c = anomalies[building_name][i].start
            i += 1
    
    index_list = energy_consumption_dfs['daily'][building_name].index
    enum_list = np.zeros(len(index_list), dtype=int)

    for a in anomalies[building_name]:
        i = index_list.get_loc(a.start)
        enum_list[i] = 1

    index_list = energy_consumption_dfs['hourly'][building_name].index

    enum_list = np.zeros(len(index_list), dtype=int)

    for a in anomalies[building_name]:
        mask = np.logical_and(a.start <= index_list, index_list < a.end)
        enum_list = enum_list + mask

    signal_list = energy_consumption_dfs['hourly'][building_name]['Totalt'].to_list() * enum_list
    signal_list = [x if x != 0 else None for x in signal_list]

    enum_list = [int(x) for x in enum_list]

    t = index_list.values.tolist()
    
    building_name_underscore = building_name.lower().replace(' ', '_').replace('.', '').replace('(', '').replace(')', '').replace('æ', 'ae').replace('ø', 'o').replace('å', 'aa')
    enum_id = f'{building_name_underscore}_anomaly_enum'
    if len(enum_id) > 40:
        enum_id = enum_id[-40:]
    enum_name = f'{building_name} anomaly enum'
    enum_metadata = SignalInfo(name=enum_name, description='', type='enum', enumValues={0: ' ', 1: 'anomaly'}, labels={"building": [building_name]})

    overlay_id = f'{building_name_underscore}_anomaly_overlay'
    if len(overlay_id) > 40:
        overlay_id = overlay_id[-40:]
    overlay_name = f'{building_name} anomaly overlay'
    overlay_metadata = SignalInfo(name=overlay_name, description='', labels={"building": [building_name]}, gapDetection="PT1H", engUnit="kWh")
    
    total_consumption_id = f'{building_name_underscore}_total_consumption'
    if len(total_consumption_id) > 40:
        total_consumption_id = total_consumption_id[-40:]
    total_consumption_name = f'{building_name} total consumption'
    total_consumption_metadata = SignalInfo(name=total_consumption_name, description='', labels={"building": [building_name]}, engUnit="kWh")
    
    total_consumption_list = energy_consumption_dfs['hourly'][building_name]['Totalt'].to_list()

    resp = clarify_client.save_signals(input_ids=[enum_id, overlay_id, total_consumption_id], signals=[enum_metadata, overlay_metadata, total_consumption_metadata])
    t = index_list.values.tolist()
    data = DataFrame(times=t, series={enum_id: enum_list, overlay_id: signal_list, total_consumption_id: total_consumption_list})
    
    return data

In [9]:
for building_name in anomalies:
    data = get_dataframe_with_anomalies(building_name)
    clarify_client.insert(data)